## Run pre-trained model on a sample image

In [14]:
from transformers import ViTImageProcessor, TFViTForImageClassification
from PIL import Image
import tensorflow as tf

img_path = 'TRAIN_IMAGES/10109_00108093-058_bb2203g0026_004_585572_fr_b2b_pan_fy22q4_sit_esg_maturity_cloud_dcf_300x250.jpg'
image = Image.open(img_path)

feature_extractor = ViTImageProcessor.from_pretrained('google/vit-base-patch16-224-in21k', 
                                                        proxies={'https': 'proxy-ir.intel.com:912'})
model = TFViTForImageClassification.from_pretrained('google/vit-base-patch16-224', 
                                                  proxies={'https': 'proxy-ir.intel.com:912'})

inputs = feature_extractor(images=image, return_tensors="tf")
outputs = model(**inputs)
logits = outputs.logits
# model predicts one of the 1000 ImageNet classes
predicted_class_idx = tf.math.argmax(logits, -1).numpy()[0]
print("Predicted class:", model.config.id2label[predicted_class_idx])


All model checkpoint layers were used when initializing TFViTForImageClassification.

All the layers of TFViTForImageClassification were initialized from the model checkpoint at google/vit-base-patch16-224.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFViTForImageClassification for predictions without further training.


Predicted class: web site, website, internet site, site


## Fine tune pretrained model

In [ ]:
from transformers import ViTImageProcessor, TFViTForImageClassification
from PIL import Image
import tensorflow as tf

import matplotlib.pyplot as plt
import numpy as np
import os
import tensorflow as tf

